# QCArchive+QCMLForge Demo with CyberShuttle

The first half of this demo shows how to use QCArchive to setup a dataset
and run computations with ease. The compute resource for this noteboook
uses Cybershuttle; however, a purely local resource demo is available under
`./demo_local.ipynb`.

The second half of this demo shows how one can consume the generated data
to train AP-Net models through QCMLForge. 

## How is this useful?
Prior to using quantum mechanical (QM) methods for applications, often computational
chemists will either consult previous studies of benchmarked methods on similar
systems, or perform the benchmarking task themselves. Then after quantifying
error, the level of theory that balances both expected error and is a reasonable
computational cost will be selected for applying to novel system(s).

The Sherrill research group has performed quite a few of these benchmarking
studies particularly for studying intermolecular interaction energies. These
interaction energies basically determine how attractive (or repulsive) molecules
behave when brought close together. A more practical definition for an
interaction energy (IE) is: 

$E_{\rm IE} = E_{\rm dimer} - E_{\rm monomerA} - E_{\rm monomerB}$.

Hence, the interaction energy is defined as the energy difference between
the dimer (when the two molecules are together) and the molecules on
their own (monomerA and monomerB in isolation).

To demonstrate how one might carryout the computational aspects of a
benchmarking study, the present notebook will provide examples of managing
datasets with QCArchive and running QM calculations prior to analyzing error
statistics with respect to reference energies. The second half of the notebook
provides examples on how you could then apply QM data towards training ML models
through QCMLForge.

In [1]:
!pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic


Loaded airavata_jupyter_magic (2.1.3.post3) 
(current runtime = local)

  %authenticate                              -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]               -- Request a runtime named <rt> with configuration <args>.
                                                Call multiple times to request multiple runtimes.
  %restart_runtime <rt>                      -- Restart runtime <rt> if it hangs. This will clear all variables.
  %stop_runtime <rt>                         -- Stop runtime <rt> when no longer needed.
  %wait_for_runtime <rt>                     -- Wait for runtime <rt> to be ready.
  %switch_runtime <rt>                       -- Switch the active runtime to <rt>. All subsequent cells will run here.
  %%run_on <rt>                              -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>                         -- Show the status of runtime <rt>.
  %copy_data source=

In [2]:
%authenticate

Output()

Authenticated.

In [3]:
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud 
%wait_for_runtime hpc_cpu --live
%switch_runtime hpc_cpu

Requesting runtime=hpc_cpu...
[NeuroData25VC1:cloud, 60 Minutes, 1 Node(s), 4 CPU(s), 0 GPU(s), 16000 MB RAM, 0 MB VRAM]
* modules=[]
* libraries=['numpy=2.2.5', 'pandas=2.2.3', 'pip', 'psycopg2=2.9.9', 'postgresql=17.4', 'pytest', 'python=3.10', 'psi4=1.9.1', 'pytorch-cpu=2.5.1', 'jupyter=1.1.1', 'requests', 'setuptools', 'torchaudio=2.5.1', 'torchvision=0.20.1', 'pytorch_geometric=2.6.1', 'pytorch_scatter=2.1.2=cpu*', 'pytorch-minimize=0.0.2', 'matplotlib=3.10.1', 'pydantic=1', 'scipy=1.15.*', 'tqdm']
* pip=['cdsg-tools==0.0.2', 'qm-tools-aw==1.4.5', 'qcmlforge==0.0.8', 'qcfractal==0.59', 'qcmanybody', 'qcfractalcompute==0.59']
* mounts=[]
Requested runtime=hpc_cpu
Request successful: runtime=hpc_cpu


Output()

local:/tmp/connection_4trcoaz6.json --> hpc_cpu:connection_4trcoaz6.json... [200]
started proc_name=hpc_cpu_kernel on rt=hpc_cpu. pid=3412
forwarding ports=[20170, 20171, 20172, 20173, 20174]
hpc_cpu:20170 -> access via 18.118.140.230:10005
hpc_cpu:20171 -> access via 18.118.140.230:10006
hpc_cpu:20172 -> access via 18.118.140.230:10007
hpc_cpu:20173 -> access via 18.118.140.230:10008
hpc_cpu:20174 -> access via 18.118.140.230:10009
18.118.140.230
started ipykernel client for hpc_cpu
Remote Jupyter kernel launched and connected for runtime=hpc_cpu.
Switched to runtime=hpc_cpu.


In [4]:
%copy_data source=local:combined_df_subset_358.pkl target=hpc_cpu:combined_df_subset_358.pkl

copying local:combined_df_subset_358.pkl to hpc_cpu:combined_df_subset_358.pkl
local:combined_df_subset_358.pkl --> hpc_cpu:combined_df_subset_358.pkl... [200]


In [5]:
import psi4
from pprint import pprint as pp
import pandas as pd
import numpy as np
from qm_tools_aw import tools
import matplotlib.pyplot as plt
# QCElemental Imports
from qcelemental.models import Molecule
import qcelemental as qcel
# Dataset Imports
from qcportal import PortalClient
from qcportal.singlepoint import SinglepointDatasetEntry, QCSpecification
from qcportal.manybody import ManybodyDatasetEntry, ManybodySpecification
from torch import manual_seed

manual_seed(42)

h2kcalmol = qcel.constants.hartree2kcalmol
print('Imports')

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Imports
cell finished on hpc_cpu.


# QCArchive Setup
QCArchive uses a PostgreSQL database for storing QM data including geometries,
energies, and properties. Furthermore, to save on compute resources, every job
generates a unique hash allowing future computations to be able to query previous
job results for avoiding re-computing calculations, unless specifically requested.

The following function initializes the QCArchive DB and QCFractal server, allowing
us to then start the services for interfacing them through python. The configurations
below will operate on a remote node through cybershuttle.

In [6]:
from qcmlforge import qca
import os

# Update these if you request non-default resources from cybershuttle.yml
max_workers = 3
cores_per_worker = 8
memory_per_worker = 12

qca.setup_qcarchive_qcfractal(
    #QCF_BASE_FOLDER=os.path.join(os.getcwd(), "qcfractal"),
    QCF_BASE_FOLDER=os.path.join("/tmp", "qcfractal"),
    start=False,
    reset=False,
    db_config={
        "name": None,
        "enable_security": "false",
        "allow_unauthenticated_read": None,
        "logfile": None,
        "loglevel": None,
        "service_frequency": 5,
        "max_active_services": None,
        "heartbeat_frequency": 60,
        "log_access": None,
        "database": {
            "base_folder": None,
            "host": None,
            "port": 5433,
            "database_name": "qca",
            "username": None,
            "password": None,
            "own": None,
        },
        "api": {
            "host": None,
            "port": 7778,
            "secret_key": None,
            "jwt_secret_key": None,
        },
    },
    resources_config={
            "update_frequency": 5,
            "cores_per_worker": cores_per_worker,
            "max_workers": max_workers,
            "memory_per_worker": memory_per_worker,
    },
    conda_env=None,
    worker_sh=None,
)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
/tmp/qcfractal

--------------------------------------------------------------------------------
Python executable:  /dev/shm/scratch/envs/9ff30a60/bin/python3.10
QCFractal version:  0.59
QCFractal alembic revision:  d5988aa750ae
pg_ctl path:  /dev/shm/scratch/envs/9ff30a60/bin/pg_ctl
PostgreSQL server version:  PostgreSQL 17.4 on x86_64-conda-linux-gnu, compiled by x86_64-conda-linux-gnu-cc (conda-forge gcc 13.3.0-2) 13.3.0, 64-bit
--------------------------------------------------------------------------------


Displaying QCFractal configuration below
--------------------------------------------------------------------------------
access_log_keep: 0
allow_unauthenticated_read: true
api:
  extra_flask_options: null
  extra_waitress_options: null
  host: localhost
  jwt_access_token_expires: 3600
  jwt_refresh_token_expires: 86400
  jwt_secret_key: nj3ZsNJk6V4WtMASCsjM6NDBgwNR4ddsJu4dg9wMgP4
  num_threads_per_worker

In [7]:
get_ipython().system = os.system
! mkdir -p qcfractal
#!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start > qcfractal/qcf_server.log &
!qcfractal-server --config=/tmp/qcfractal/qcfractal_config.yaml start > qcfractal/qcf_server.log &

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
0
cell finished on hpc_cpu.


In [8]:
!qcfractal-compute-manager --config=/tmp/qcfractal/resources.yml &
#!qcfractal-compute-manager --config=`pwd`/qcfractal/resources.yml &

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
0
cell finished on hpc_cpu.


In [9]:
# Establish client connection
client = PortalClient("http://localhost:7778", verify=False)
print(client)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
PortalClient(server_name='QCFractal Server', address='http://localhost:7778/', username='None')
cell finished on hpc_cpu.


# QCArchive single point example with Psi4

QCArchive/QCFractal can be used to run several QM softwares, but the focus of
the present notebook will highlight the popular, open-source Psi4 program. 

As a simple example, we can specify a water dimer geometry and then
create a singlepoint energy entry into the database with the client.

In [10]:
# Running a single job
mol = Molecule.from_data(
    """
     0 1
     O  -1.551007  -0.114520   0.000000
     H  -1.934259   0.762503   0.000000
     H  -0.599677   0.040712   0.000000
     --
     0 1
     O   1.350625   0.111469   0.000000
     H   1.680398  -0.373741  -0.758561
     H   1.680398  -0.373741   0.758561

     units angstrom
     no_reorient
     symmetry c1
"""
)

client.add_singlepoints(
    [mol],
    "psi4",
    driver="energy",
    method="b3lyp",
    basis="aug-cc-pvdz",
    keywords={"scf_type": "df", "e_convergence": 6, "freeze_core": True},
    tag="local",
)


executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
[2025-05-07 07:32:11 UTC]     INFO: qcfractalcompute.config: Reading configuration data from /tmp/qcfractal/resources.yml
(InsertMetadata(error_description=None, errors=[], inserted_idx=[], existing_idx=[0]),
 [1])
cell finished on hpc_cpu.


Process Process-1:
Traceback (most recent call last):
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/qcfractal/flask_app/waitress_app.py", line 40, in run
    serve(
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/waitress/__init__.py", line 13, in serve
    server = _server(app, **kw)
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/waitress/server.py", line 78, in create_server
    last_serv = TcpWSGIServer(
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/waitress/server.py", line 243, in __init__
    self.bind_server_socket()
  File "/dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/waitress/server.py", line 364, in bind_server_soc

## Query Results

In [11]:
recs = client.query_records(
    record_id=[1]
)
for rec in recs:
    print(rec.dict)
    #print(rec.properties['return_result'])

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
<bound method BaseModel.dict of SinglepointRecord(id=1, record_type='singlepoint', is_service=False, properties={'pe energy': 0.0, 'scf dipole': [1.01443376121478, 0.030397167953628346, 4.728134984227639e-12], 'calcinfo_nmo': 82, 'dft xc energy': -15.09393332583478, 'return_energy': -152.8965873433082, 'return_result': -152.8965873433082, 'scf_xc_energy': -15.09393332583478, 'calcinfo_natom': 6, 'calcinfo_nbeta': 10, 'current dipole': [1.01443376121478, 0.030397167953628346, 4.728134984227639e-12], 'current energy': -152.8965873433082, 'return_hessian': None, 'scf_iterations': 8, 'calcinfo_nalpha': 10, 'calcinfo_nbasis': 82, 'dft vv10 energy': 0.0, 'return_gradient': None, 'dft total energy': -152.89658734330817, 'scf_total_energy': -152.8965873433082, 'scf_dipole_moment': [1.01443376121478, 0.030397167953628346, 4.728134984227639e-12], 'scf_total_hessian': None, 'scf total energies': [-152.1832917902242, -152.548098

# Dataset Example
While one could use the above logic for all calculations, QCArchive provides
dataset options to make managing data for a specific application substantially
easier. 

Prior to creating the QCArchive datasets, we will first load in data from a 
recently submitted Sherrill work [Levels of SAPT II](https://chemrxiv.org/engage/chemrxiv/article-details/67fe885f6e70d6fb2e033804). To faciliate insertion
into the DB, we will create QCElemental Molecule objects in the pickle file
and extract specific columns to store as variables for usage below. 

In [12]:
# Creating a QCArchive Dataset...
# Load in a dataset from a recent Sherrill work (Levels of SAPT II)
df_LoS = pd.read_pickle("./combined_df_subset_358.pkl")
print(df_LoS[['Benchmark', 'SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz', 'MP2 IE atz', 'SAPT0 TOTAL ENERGY adz' ]])

# Limit to 100 molecules with maximum of 16 atoms to keep computational cost down
df_LoS['size'] = df_LoS['atomic_numbers'].apply(lambda x: len(x))
df_LoS = df_LoS[df_LoS['size'] <= 16]
df_LoS = df_LoS.sample(50, random_state=42, axis=0).copy()
# df_LoS = df_LoS.sample(50, random_state=42, axis=0).copy()
df_LoS.reset_index(drop=True, inplace=True)
print(df_LoS['size'].describe())

# Create QCElemntal Molecules to generate the dataset
def qcel_mols(row):
    """
    Convert the row to a qcel molecule
    """
    atomic_numbers = [row['atomic_numbers'][row['monAs']], row['atomic_numbers'][row['monBs']]]
    coords = [row['coordinates'][row['monAs']], row['coordinates'][row['monBs']]]
    cm = [
        [row['monA_charge'], row['monA_multiplicity']],
        [row['monB_charge'], row['monB_multiplicity']],
     ]
    return tools.convert_pos_carts_to_mol(atomic_numbers, coords, cm)
df_LoS['qcel_molecule'] = df_LoS.apply(qcel_mols, axis=1)
geoms = df_LoS['qcel_molecule'].tolist()
ref_IEs = df_LoS['Benchmark'].tolist()
sapt0_adz = (df_LoS['SAPT0 TOTAL ENERGY adz'] * h2kcalmol).tolist()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
     Benchmark SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz  MP2 IE atz  \
0      -10.248                         -0.016681   -0.015629   
1      -15.245                         -0.024763   -0.023012   
2       -3.517                         -0.005637   -0.005608   
3       -0.127                         -0.000187   -0.000194   
4       -8.990                         -0.014655   -0.013687   
..         ...                               ...         ...   
353     -4.390                         -0.007196   -0.006835   
354     -1.130                         -0.001489   -0.002395   
355     -0.260                         -0.000432   -0.000450   
356     -5.740                         -0.009198   -0.008974   
357     -3.120                         -0.004909   -0.005518   

     SAPT0 TOTAL ENERGY adz  
0                 -0.018254  
1                 -0.027620  
2                 -0.005920  
3                 -0.000192  
4          

## Singlepoint Dataset

First, to demonstrate how you could use this data for singlepoint energies/properties like the water
dimer example above, we will create a singlepoint dataset and slot in our geometry data. 

In [13]:
# Create client dataset
ds_name = 'LoS-singlepoint'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name not in client_datasets:
    ds = client.add_dataset("singlepoint", ds_name,
                            f"Dataset to contain {ds_name}")
    print(f"Added {ds_name} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        extras = {
            "name": 'LoS-' + str(idx),
            "idx": idx,
        }
        mol = Molecule.from_data(mol.dict(), extras=extras)
        ent = SinglepointDatasetEntry(name=extras['name'], molecule=mol)
        entry_list.append(ent)
    ds.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds = client.get_dataset("singlepoint", ds_name)
    print(f"Found {ds_name} dataset, using this instead")

print(ds)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Found LoS-singlepoint dataset, using this instead
id=1 dataset_type='singlepoint' name='LoS-singlepoint' description='Dataset to contain LoS-singlepoint' tagline='' tags=[] group='default' visibility=True provenance={} default_tag='*' default_priority=<PriorityEnum.normal: 1> owner_user=None owner_group=None metadata={} extras={} contributed_values_=None attachments_=None auto_fetch_missing=True
cell finished on hpc_cpu.


# Create QCSpecification and submit computations
Now that the dataset has been created with our molecular systems, we will
specify a level of theory (method/basis set) for running our calculations. We
will name our specification "psi4/SAPT0/cc-pvdz" to easily label this
specification's level of theory and program. This name will be used later
during data collection and analysis. Although many QM methods that use a
singlepoint energy would return a total energy, SAPT is a specialized method
that directly computes the interaction energy between our monomers.

In [14]:
# SAPT0 Example
method, basis = "SAPT0", "cc-pvdz"

# Set the QCSpecification (QM interaction energy in our case)
spec = QCSpecification(
    program="psi4",
    driver="energy",
    method=method,
    basis=basis,
    keywords={
        "scf_type": "df",
    },
)
ds.add_specification(name=f"psi4/{method}/{basis}", specification=spec)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
InsertMetadata(error_description=None, errors=[], inserted_idx=[], existing_idx=[0])
cell finished on hpc_cpu.


In [15]:
# Run the computations
ds.submit()
print(f"Submitted {ds_name} dataset")

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Submitted LoS-singlepoint dataset
cell finished on hpc_cpu.


In [16]:
ds.status()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
{'psi4/SAPT0/cc-pvdz': {<RecordStatusEnum.complete: 'complete'>: 50}}
cell finished on hpc_cpu.


## Manybody Dataset - typical way to compute interaction energies with most QM methods

While SAPT0 returns an interaction energy from a singlepoint energy, most other QM
methods like HF, MP2, CCSD, or DFT would return a total dimer energy ($E_{\rm dimer}$). To
get QCArchive to run the energies of the dimer, monomerA, and monomerB and return the 
energy difference as above for $E_{\rm IE}$, we can use the ManybodyDataset. The setup is
similar to the singlepoint dataset above, but using different objects.

In [17]:
# Create client dataset
ds_name_mb = 'LoS-manybody'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name_mb not in client_datasets:
    print("Setting up new dataset:", ds_name_mb)
    ds_mb = client.add_dataset("manybody", ds_name_mb,
                            f"Dataset to contain {ds_name_mb}")
    print(f"Added {ds_name_mb} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        ent = ManybodyDatasetEntry(name=f"LoS-IE-{idx}", initial_molecule=mol)
        entry_list.append(ent)
    ds_mb.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds_mb = client.get_dataset("manybody", ds_name_mb)
    print(f"Found {ds_name_mb} dataset, using this instead")

print(ds_mb)

# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=2, delete_records=True)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Found LoS-manybody dataset, using this instead
id=2 dataset_type='manybody' name='LoS-manybody' description='Dataset to contain LoS-manybody' tagline='' tags=[] group='default' visibility=True provenance={} default_tag='*' default_priority=<PriorityEnum.normal: 1> owner_user=None owner_group=None metadata={} extras={} contributed_values_=None attachments_=None auto_fetch_missing=True
cell finished on hpc_cpu.


In [18]:
ds_mb.status()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
{'psi4/hf/6-31g*': {<RecordStatusEnum.complete: 'complete'>: 1,
  <RecordStatusEnum.running: 'running'>: 19,
  <RecordStatusEnum.waiting: 'waiting'>: 30}}
cell finished on hpc_cpu.


# Specify level of theory for Manybody dataset
Similar to above, we can iterate through different levels of theory, create
QCSpecifications, and submit the dataset computations. The major difference here
is that now the "levels" must be specified for the ManybodySpecification. These
"levels" refer to what level of theory and options we want to run our dimer (2)
and monomer (1) computations with. 

In [19]:
# Set multiple levels of theory - you can add/remove levels as you desire.
# Computational scaling will get quite expensive with better methods and larger
# basis sets

methods = [
    'hf', # 'svwn', # 'pbe', 
]
basis_sets = [
    '6-31g*'
]

for method in methods:
    for basis in basis_sets:
        # Set the QCSpecification (QM interaction energy in our case)
        qc_spec_mb = QCSpecification(
            program="psi4",
            driver="energy",
            method=method,
            basis=basis,
            keywords={
                "d_convergence": 8,
                "scf_type": "df",
            },
        )

        spec_mb = ManybodySpecification(
            program='qcmanybody',
            bsse_correction=['cp'],
            levels={
                1: qc_spec_mb,
                2: qc_spec_mb,
            },
        )
        print("spec_mb", spec_mb)

        ds_mb.add_specification(name=f"psi4/{method}/{basis}", specification=spec_mb)

        # Run the computations
        ds_mb.submit()
        print(f"Submitted {ds_name} dataset")

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
spec_mb program='qcmanybody' levels={1: QCSpecification(program='psi4', driver=<SinglepointDriver.energy: 'energy'>, method='hf', basis='6-31g*', keywords={'d_convergence': 8, 'scf_type': 'df'}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<NativeFilesProtocolEnum.none: 'none'>)), 2: QCSpecification(program='psi4', driver=<SinglepointDriver.energy: 'energy'>, method='hf', basis='6-31g*', keywords={'d_convergence': 8, 'scf_type': 'df'}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<NativeFilesProtocolEnum.none: 'none'>))} bsse_correction=[<BSSECorrectionEnum.cp: 'cp'>] keywords=ManybodyKeywords(return_total_data=False) protocols={}
S

# Dataset Status
We can re-execute the following cell to see how many computations have completed and how many are running/waiting.

In [20]:
pp(ds.status())
pp(ds_mb.status())

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
{'psi4/SAPT0/cc-pvdz': {<RecordStatusEnum.complete: 'complete'>: 50}}
{'psi4/hf/6-31g*': {<RecordStatusEnum.complete: 'complete'>: 1,
                    <RecordStatusEnum.running: 'running'>: 19,
                    <RecordStatusEnum.waiting: 'waiting'>: 30}}
cell finished on hpc_cpu.


# PAUSE HERE TO WAIT FOR COMPUTATIONS TO FINISH

In [37]:
import time

def wait_for_finish(tasks):
    processing = True
    while processing:
        processing = False
        for name, d in tasks.items():
            for k, v in d.status().items():
                if not all(i == "complete" for i in v.keys()):
                    processing = True
            print(name, "RUNNING" if processing else "DONE")
        if processing:
            time.sleep(15)

wait_for_finish({"ds": ds, "ds_mb": ds_mb})

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
ds DONE
ds_mb DONE
cell finished on hpc_cpu.


# Data Assembly

NOTE: While you can execute the following blocks before all computations are
complete, it is recommended to wait until all computations are complete to
continue.

QCArchive allows use to write functions that operate on each dataset entry to return exactly what information
we want from every record through the `compile_values` method. The method returns a
pandas dataframe containing our QCSpecification names and our specified columns for each entry 
in our assemble data functions. Here is the Singlepoint data assembly for SAPT calculations. 

In [38]:
# Singlepoint data assemble
def assemble_singlepoint_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    sapt_energies = np.array([np.nan, np.nan, np.nan, np.nan, np.nan])
    sapt_energies[0] = qcvars['sapt total energy']
    sapt_energies[1] = qcvars['sapt elst energy']
    sapt_energies[2] = qcvars['sapt exch energy']
    sapt_energies[3] = qcvars['sapt ind energy']
    sapt_energies[4] = qcvars['sapt disp energy']
    return (
        record.molecule,
        record.molecule.atomic_numbers,
        record.molecule.geometry * qcel.constants.bohr2angstroms,
        int(record.molecule.molecular_charge),
        record.molecule.molecular_multiplicity,
        sapt_energies,
    )

def assemble_singlepoint_data_value_names():
    return [
        'qcel_molecule',
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity",
        "SAPT Energies",
    ]

df = ds.compile_values(
    value_call=assemble_singlepoint_data,
    value_names=assemble_singlepoint_data_value_names(),
    unpack=True,
)
pp(df.columns.tolist())
df_sapt0 = df['psi4/SAPT0/cc-pvdz']

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
[('psi4/SAPT0/cc-pvdz', 'qcel_molecule'),
 ('psi4/SAPT0/cc-pvdz', 'Z'),
 ('psi4/SAPT0/cc-pvdz', 'R'),
 ('psi4/SAPT0/cc-pvdz', 'TQ'),
 ('psi4/SAPT0/cc-pvdz', 'molecular_multiplicity'),
 ('psi4/SAPT0/cc-pvdz', 'SAPT Energies')]
cell finished on hpc_cpu.


Below is the assemble data function for our Manybody dataset results:

In [39]:
def assemble_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    CP_IE = qcvars['results']['cp_corrected_interaction_energy'] * h2kcalmol
    NOCP_IE = qcvars['results'].get('nocp_corrected_interaction_energy', np.nan) * h2kcalmol
    return (
    record.initial_molecule,
    CP_IE,
    NOCP_IE,
    record.initial_molecule.atomic_numbers,
    record.initial_molecule.geometry * qcel.constants.bohr2angstroms,
    int(record.initial_molecule.molecular_charge),
    record.initial_molecule.molecular_multiplicity,
    )

def assemble_data_value_names():
    return [
        'qcel_molecule',
        "CP_IE",
        "NOCP_IE",
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity"
    ]

df_mb = ds_mb.compile_values(
    value_call=assemble_data,
    value_names=assemble_data_value_names(),
    unpack=True,
)

pp(df_mb.columns.tolist())

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
[('psi4/hf/6-31g*', 'qcel_molecule'),
 ('psi4/hf/6-31g*', 'CP_IE'),
 ('psi4/hf/6-31g*', 'NOCP_IE'),
 ('psi4/hf/6-31g*', 'Z'),
 ('psi4/hf/6-31g*', 'R'),
 ('psi4/hf/6-31g*', 'TQ'),
 ('psi4/hf/6-31g*', 'molecular_multiplicity')]
cell finished on hpc_cpu.


Now we can pull together this data for analyzing our results. Our group has some
common plotting scripts operating on pandas Dataframes, so we will convert
our computed data into a format that is compatable with these scripts.

In [42]:
from cdsg_plot import error_statistics

df_sapt0['sapt0 total energes'] = df_sapt0.apply(lambda x: x['SAPT Energies'][0] * h2kcalmol, axis=1)
df_plot = pd.DataFrame(
    {
        "qcel_molecule": df_mb["psi4/hf/6-31g*"]["qcel_molecule"],
        "HF/6-31G*": df_mb["psi4/hf/6-31g*"]["CP_IE"],
        'SAPT0/cc-pvdz': df_sapt0['sapt0 total energes'].values,
        # "svwn/6-31G*": df_mb["psi4/svwn/6-31g*"]["CP_IE"],
        # "PBE/6-31G*": df_mb["psi4/pbe/6-31g*"]["CP_IE"],
    }
)
# print(df_plot)
id = [int(i[7:]) for i in df_plot.index]
df_plot['id'] = id
df_plot.sort_values(by='id', inplace=True, ascending=True)
df_plot['reference'] = ref_IEs
df_plot['SAPT0/aug-cc-pvdz'] = sapt0_adz
df_plot['HF/6-31G* error'] = (df_plot['HF/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['PBE/6-31G* error'] = (df_plot['PBE/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['svwn/6-31G* error'] = (df_plot['svwn/6-31G*'] - df_plot['reference']).astype(float)
df_plot['SAPT0/cc-pvdz error'] = (df_plot['SAPT0/cc-pvdz'] - df_plot['reference']).astype(float)
df_plot['SAPT0/aug-cc-pvdz error'] = (df_plot['SAPT0/aug-cc-pvdz'] - df_plot['reference']).astype(float)
# print(df_plot[['HF/6-31G*', 'SAPT0/cc-pvdz', 'reference', "SAPT0/aug-cc-pvdz"]])
df_plot = df_plot.dropna(subset=['qcel_molecule', 'HF/6-31G*', 'SAPT0/cc-pvdz', 'SAPT0/aug-cc-pvdz'])
print(df_plot[['HF/6-31G* error', 'SAPT0/cc-pvdz error', "SAPT0/aug-cc-pvdz error"]].describe())

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
       HF/6-31G* error  SAPT0/cc-pvdz error  SAPT0/aug-cc-pvdz error
count        50.000000            50.000000                50.000000
mean          1.590681            -0.593706                -1.471363
std           2.368003             1.447307                 2.061593
min          -3.161351            -6.499656                -9.134743
25%           0.159828            -0.511685                -1.884289
50%           0.656320            -0.039749                -0.514994
75%           2.049042             0.128829                -0.233319
max           9.469009             0.975165                -0.003784
cell finished on hpc_cpu.


/home/exouser/cybershuttle/scratch/tmp/ipykernel_3414/1492624606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sapt0['sapt0 total energes'] = df_sapt0.apply(lambda x: x['SAPT Energies'][0] * h2kcalmol, axis=1)


# Plotting the interaction energy errors
The function below will plot violin plots of the error distributions of the
approximate level of theory and our reference. For this dataset, the reference
energies are estimated CCSD(T)/CBS interaction energies provided by the Levels
of SAPT II Supplemental Information.

In [45]:
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
    },
    output_filename="S22-IE.png",
    figure_size=(6, 6),
    x_label_fontsize=16,
    ylim=(-15, 15),
    rcParams={},
    usetex=False,
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)
plt.show()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Plotting S22-IE.png
(-15, 15)
lower_bound = -15, upper_bound = 20, inc = 5
S22-IE_violin.png
<Figure size 3840x2880 with 0 Axes>
<Figure size 600x600 with 0 Axes>
cell finished on hpc_cpu.


# QCMLForge
The previous sections conclude the benchmarking workflow; however, the remaining blocks in this
notebook focus on using this QM data to train models and run inference to compare against QM methods. 

## Interaction energy ML model inference
A specialized atom-pairwise neural network
([AP-Net](https://pubs.rsc.org/en/content/articlehtml/2024/sc/d4sc01029a))
architecture developed in the Sherrill group has been trained on 1.6 million
dimers to predict SAPT0/aug-cc-pV(D+d)Z interaction energies. The model has been
re-implemented and re-trained in PyTorch in QCMLForge for extending in the
future. Below will take a pre-trained AP-Net2 model and predict energies on our
current dimer dataset.  Note that the AP-Net2 architecture relies on first an
AtomModel for predicting multipoles and intramolecular features, prior to more
message passing neural networks for predicting SAPT component energies. As such,
we must load pre-trained AtomModel weights and APNet2Model weights.

In [46]:
import apnet_pt
from apnet_pt.AtomPairwiseModels.apnet2 import APNet2Model
from apnet_pt.AtomModels.ap2_atom_model import AtomModel

atom_model = AtomModel().set_pretrained_model(model_id=0)
ap2 = APNet2Model(atom_model=atom_model.model).set_pretrained_model(model_id=0)
ap2.atom_model = atom_model.model
print(df_plot['qcel_molecule'].tolist())
apnet2_ies_predicted = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16
)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
running on the CPU
running on the CPU
self.dataset=None
[Molecule(name='C2H4O4', formula='C2H4O4', hash='c5abb54'), Molecule(name='CH7NO', formula='CH7NO', hash='5ec955b'), Molecule(name='C3H9NO2', formula='C3H9NO2', hash='8682f32'), Molecule(name='C6H8O', formula='C6H8O', hash='eaedeb0'), Molecule(name='C4H8O4', formula='C4H8O4', hash='35b6c18'), Molecule(name='C4H8O4', formula='C4H8O4', hash='85f67af'), Molecule(name='H4O2', formula='H4O2', hash='80c4390'), Molecule(name='C2H6N2O2', formula='C2H6N2O2', hash='084885d'), Molecule(name='C3H9NO2', formula='C3H9NO2', hash='b28b356'), Molecule(name='C2H4O4', formula='C2H4O4', hash='f55cc2e'), Molecule(name='CH7NO', formula='CH7NO', hash='682fa43'), Molecule(name='FH2O', formula='FH2O-', hash='d69cf34'), Molecule(name='C2H10N2', formula='C2H10N2', hash='9f0ed39'), Molecule(name='C6H8O', formula='C6H8O', hash='849097c'), Molecule(name='C2H4O4', formula='C2H4O4', hash='55b0

In [47]:
# AP-Net2 IE
df_plot['APNet2'] = np.sum(apnet2_ies_predicted, axis=1)
df_plot['APNet2 error'] = (df_plot['APNet2'] - df_plot['reference']).astype(float)
#print(df_plot.sort_values(by='APNet2 error', ascending=True)[['APNet2', 'reference']])
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
    },
    output_filename="S22-IE-AP2.png",
    rcParams={},
    usetex=False,
    figure_size=(4, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)
plt.show()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Plotting S22-IE-AP2.png
S22-IE-AP2_violin.png
<Figure size 3840x2880 with 0 Axes>
<Figure size 400x400 with 0 Axes>
cell finished on hpc_cpu.


# Transfer Learning
To demonstrate transfer learning, we can use our reference data to then tune our
pre-trained SAPT0 model to predict the CCSD(T)/CBS energies. Then we will
re-plot, evaluating the model performance. Note that in any application, you
should certainly use much more data and careful train/test set selection than
what is done in this notebook. This is purely meant for demonstraion purposes
only.

In [48]:
# Training models on new QM data: Transfer Learning

from apnet_pt import pairwise_datasets

ds2 = pairwise_datasets.apnet2_module_dataset(
    root="data_dir",
    spec_type=None,
    atom_model=atom_model,
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=[np.array([i]) for i in df_plot['reference'].tolist()],
    skip_compile=True,
    force_reprocess=True,
    atomic_batch_size=8,
    prebatched=False,
    in_memory=True,
    batch_size=4,
)
print(ds2)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Received 50 QCElemental molecules with energy labels
Processing directly from provided QCElemental molecules...
Processing 50 dimers from provided QCElemental molecules...
Creating data objects...
len(RAs)=50, self.atomic_batch_size=8, self.batch_size=4
0/50, 0.04s, 0.04s
8/50, 0.03s, 0.07s
16/50, 0.03s, 0.10s


Processing...


24/50, 0.03s, 0.13s
32/50, 0.03s, 0.16s
40/50, 0.03s, 0.19s
48/50, 0.01s, 0.20s
Processing directly from provided QCElemental molecules...
Processing 50 dimers from provided QCElemental molecules...
Creating data objects...
len(RAs)=50, self.atomic_batch_size=8, self.batch_size=4
0/50, 0.03s, 0.03s
8/50, 0.03s, 0.06s
16/50, 0.03s, 0.09s
24/50, 0.03s, 0.12s
32/50, 0.03s, 0.15s
40/50, 0.03s, 0.18s
48/50, 0.01s, 0.19s
self.root='data_dir', self.spec_type=None, self.in_memory=True
apnet2_module_dataset(50)
cell finished on hpc_cpu.


Done!
Processing...
Done!


In [49]:
# Transfer Learning APNet2 model on computed QM data
ap2.train(
    dataset=ds2,
    n_epochs=50,
    transfer_learning=True,
    skip_compile=True,
    model_path="apnet2_transfer_learning.pt",
    split_percent=0.8,
)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Saving training results to...
apnet2_transfer_learning.pt
~~ Training APNet2Model ~~
    Training on 40 samples, Testing on 10 samples

Network Hyperparameters:
  self.model.n_message=3
  self.model.n_neuron=128
  self.model.n_embed=8
  self.model.n_rbf=8
  self.model.r_cut=5.0
  self.model.r_cut_im=8.0

Training Hyperparameters:
  n_epochs=50
  lr=0.0005

  lr_decay=None

  batch_size=4
Running single-process training
                                       Total
  (Pre-training) (0.67   s)  MAE:   1.328/3.007  
  EPOCH:    0 (0.68   s)  MAE:   1.408/4.212    
  EPOCH:    1 (0.62   s)  MAE:   2.681/6.722    
  EPOCH:    2 (0.72   s)  MAE:   1.634/1.754   *
  EPOCH:    3 (0.70   s)  MAE:   2.035/1.696   *
  EPOCH:    4 (0.72   s)  MAE:   0.660/1.730   *
  EPOCH:    5 (0.69   s)  MAE:   0.635/1.151   *
  EPOCH:    6 (0.66   s)  MAE:   0.591/2.942    
  EPOCH:    7 (0.67   s)  MAE:   0.780/1.062    
  EPOCH:    8 (0.65 

In [51]:
# AP-Net2 IE
apnet2_ies_predicted_transfer = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16,
)
df_plot['APNet2 transfer'] = np.sum(apnet2_ies_predicted_transfer, axis=1)
df_plot['APNet2 transfer error'] = (df_plot['APNet2 transfer'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
    },
    output_filename="S22-IE-AP2-tf.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)
plt.show()

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Plotting S22-IE-AP2-tf.png
S22-IE-AP2-tf_violin.png
<Figure size 3840x2880 with 0 Axes>
<Figure size 600x400 with 0 Axes>
cell finished on hpc_cpu.


## $\Delta$ Models
Another option available in QCMLForge is to train $\Delta {\rm APNet2}$ models 
to predict the difference between one level of theory and another. This
example highlights learning how to go from our computed HF/6-31G* IEs
to our CCSD(T)/CBS energies.

In [52]:
from apnet_pt.pt_datasets.dapnet_ds import dapnet2_module_dataset_apnetStored

delta_energies = df_plot['HF/6-31G* error'].tolist()

# Only operates in pre-batched mode
ds_dap2 = dapnet2_module_dataset_apnetStored(
    root="data_dir",
    r_cut=5.0,
    r_cut_im=8.0,
    spec_type=None,
    max_size=None,
    force_reprocess=True,
    batch_size=2,
    num_devices=1,
    skip_processed=False,
    skip_compile=True,
    print_level=2,
    in_memory=True,
    m1="HF/6-31G*",
    m2="CCSD(T)/CBS",
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=delta_energies,
)
print(ds_dap2)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
Received 50 QCElemental molecules with energy labels
running on the CPU
running on the CPU
Loading pre-trained APNet2_MPNN model from /dev/shm/scratch/envs/9ff30a60/lib/python3.10/site-packages/apnet_pt/models/ap2_ensemble/ap2_0.pt
self.dataset=None
raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Creating data objects...
len(qcel_mols)=50, self.batch_size=2


Processing...
Done!
Processing...


raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Creating data objects...
len(qcel_mols)=50, self.batch_size=2
self.root='data_dir', self.spec_type=None, self.in_memory=True
raw_path: data_dir/raw/splinter_spec1.pkl
Loading dimers...
Saving to data_dir/processed_delta/targets_HF6-31G_to_CCSD_LP_T_RP_CBS.pt
dapnet2_module_dataset_apnetStored(25)
cell finished on hpc_cpu.


Done!


In [53]:
from apnet_pt.AtomPairwiseModels.dapnet2 import dAPNet2Model

dap2 = dAPNet2Model(
    atom_model=AtomModel().set_pretrained_model(model_id=0),
    apnet2_model=APNet2Model().set_pretrained_model(model_id=0).set_return_hidden_states(True),
)
dap2.train(
    ds_dap2,
    n_epochs=50,
    skip_compile=True,
    split_percent=0.6,
)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
running on the CPU
running on the CPU
self.dataset=None
running on the CPU
None
Saving training results to...
None
~~ Training APNet2Model ~~
    Training on 15 samples, Testing on 10 samples

Network Hyperparameters:
  self.model.n_neuron=128

Training Hyperparameters:
  n_epochs=50
  lr=0.0005

  lr_decay=None

  batch_size=1
Running single-process training
num_workers = 4
                                       Energy
  (Pre-training) (0.34   s)  MAE:   1.500/2.153  
  EPOCH:    0 (0.38   s)  MAE:   1.106/1.238   *
  EPOCH:    1 (0.39   s)  MAE:   0.881/1.044   *
  EPOCH:    2 (0.37   s)  MAE:   0.803/1.062    
  EPOCH:    3 (0.37   s)  MAE:   0.798/0.971    
  EPOCH:    4 (0.38   s)  MAE:   0.710/0.970    
  EPOCH:    5 (0.37   s)  MAE:   0.694/0.924   *
  EPOCH:    6 (0.32   s)  MAE:   0.659/0.872   *
  EPOCH:    7 (0.38   s)  MAE:   0.598/0.849    
  EPOCH:    8 (0.37   s)  MAE:   0.595/0.808   *
  EPOCH:    9 (

In [54]:
dAPNet2_ies_predicted_transfer = dap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=2,
)
df_plot['dAPNet2'] = dAPNet2_ies_predicted_transfer
df_plot['HF/6-31G*-dAPNet2'] = df_plot['HF/6-31G*'] - df_plot['dAPNet2']
print(df_plot[['dAPNet2', 'HF/6-31G*', 'HF/6-31G*-dAPNet2',  'reference']])
df_plot['dAPNet2 error'] = (df_plot['HF/6-31G*-dAPNet2'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
        "dAPNet2 HF/6-31G* to CCSD(T)/CBS": "dAPNet2 error",
    },
    output_filename="S22-IE-AP2-dAP2.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

executing cell on hpc_cpu...
waiting for cell to finish on hpc_cpu...
            dAPNet2  HF/6-31G* HF/6-31G*-dAPNet2  reference
entry                                                      
LoS-IE-0   3.034600  -15.29702         -18.33162    -18.075
LoS-IE-1   0.441592  -4.790555         -5.232147     -5.220
LoS-IE-2   0.954113      -3.52         -4.474113     -4.700
LoS-IE-3   0.181786  -0.441187         -0.622973     -0.420
LoS-IE-4   0.504614  -8.814418         -9.319032     -9.210
LoS-IE-5   4.960127 -14.037943         -18.99807    -18.790
LoS-IE-6   0.551140   -4.02006         -4.571199     -4.570
LoS-IE-7   0.237287  -3.383401         -3.620688     -3.528
LoS-IE-8   1.289526  -6.188112         -7.477638     -7.460
LoS-IE-9   4.748248  -14.98262        -19.730867    -19.650
LoS-IE-10 -0.047835   -1.21354         -1.165706     -1.200
LoS-IE-11 -3.032284 -35.661351        -32.629067    -32.500
LoS-IE-12  0.292892  -0.971568          -1.26446     -1.310
LoS-IE-13  1.297276  -1.773877

In [55]:
%stop_runtime hpc_cpu

Terminated runtime=hpc_cpu. state={'experimentId': 'CS_Agent_6036b61f-3f55-4747-ba0f-d8661f0f48cd', 'terminated': True}


# The end...

This notebook has been designed to help demonstrate how QCArchive can be used to
compute QM properties through cybershuttle, and then leverage that data for
analysis or training AP-Net based models through QCMLForge. 

If you have any questions about this notebook or used packages, please feel
free to reach out to me at awallace43@gatech.edu!